In [11]:
from __future__ import print_function
from ortools.linear_solver import pywraplp
import time

def main():
  start = time.time()
  cost = [[100000, 76000, 75000, 70000, 50000, 74000],
          [35000, 85000, 55000, 65000, 48000, 101000],
          [125000, 95000, 90000, 105000, 59000, 120000],
          [45000, 110000, 95000, 115000, 104000, 83000],
          [60000, 105000, 80000, 75000, 59000, 62000],
          [45000, 65000, 110000, 95000, 47000, 310000],
          [38000, 51000, 107000, 41000, 69000, 99000],
          [47000, 85000, 57000, 71000, 92000, 77000],
          [39000, 63000, 97000, 49000, 118000, 56000],
          [47000, 101000, 71000, 60000, 88000, 109000],
          [17000, 39000, 103000, 64000, 61000, 92000],
          [101000, 45000, 83000, 59000, 92000, 27000]]

  num_tasks = len(cost[1])
  # Allowed groups of workers:
  group1 =  [[2, 3],       # Subgroups of workers 0 - 3
             [1, 3],
             [1, 2],
             [0, 1],
             [0, 2]]

  group2 =  [[6, 7],       # Subgroups of workers 4 - 7
             [5, 7],
             [5, 6],
             [4, 5],
             [4, 7]]

  group3 =  [[10, 11],     # Subgroups of workers 8 - 11
             [9, 11],
             [9, 10],
             [8, 10],
             [8, 11]]

  allowed_groups = []

  for i in range(len(group1)):
    for j in range(len(group2)):
      for k in range(len(group3)):
        allowed_groups.append(group1[i] + group2[j] + group3[k])
  min_val = 1e6
  total_time = 0

  for i in range(len(allowed_groups)):
    group = allowed_groups[i]
    res = assignment(cost, group)
    solver_tmp = res[0]
    x_tmp = res[1]
    total_time = total_time + solver_tmp.WallTime()

    if solver_tmp.Objective().Value() < min_val:
      min_val = solver_tmp.Objective().Value()
      min_index = i
      min_solver = solver_tmp
      min_x = x_tmp
      min_group = group

  print('Minimum cost = ', min_val)
  print()
  for i in min_group:
    for j in range(num_tasks):
      if min_x[i, j].solution_value() > 0:
        print('Tugas', j,' kepada pekerja', i, ' biaya = Rp.', cost[i][j],',-')
  print()
  end = time.time()
  print("Time = ", round(end - start, 4), "seconds")

def assignment(cost, group):
  # Solve assignment problem for given group of workers.
  num_tasks = len(cost[1])
  # Clear values in x
  solver = None
  # Instantiate a mixed-integer solver
  solver = pywraplp.Solver('AssignmentProblemGroups',
                           pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
  x = None
  x = {}

  for i in group:
    for j in range(num_tasks):
      x[i, j] = solver.IntVar(0, 1, 'x[%i,%i]' % (i, j))

  # Each worker is assigned to exactly one task.

  for i in group:
    solver.Add(solver.Sum([x[i, j] for j in range(num_tasks)]) <= 1)

  # Each task is assigned to at least one worker.

  for j in range(num_tasks):
    solver.Add(solver.Sum([x[i, j] for i in group]) >= 1)
  # Objective
  solver.Minimize(solver.Sum([cost[i][j] * x[i,j] for i in group
                                                  for j in range(num_tasks)]))
  solver.Solve()
  res = [solver, x]
  return res

if __name__ == '__main__':
  main()

Minimum cost =  249000.0

Tugas 4  kepada pekerja 0  biaya = Rp. 50000 ,-
Tugas 0  kepada pekerja 1  biaya = Rp. 35000 ,-
Tugas 3  kepada pekerja 6  biaya = Rp. 41000 ,-
Tugas 2  kepada pekerja 7  biaya = Rp. 57000 ,-
Tugas 1  kepada pekerja 10  biaya = Rp. 39000 ,-
Tugas 5  kepada pekerja 11  biaya = Rp. 27000 ,-

Time =  0.9035 seconds
